# Grid2x2 using RLlib

In [1]:
import csv
import os

import ray
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility
from ray.tune.registry import register_env

from envs import MultiAgentSumoEnv
from observation import Grid2x2ObservationFunction
from reward_functions import combined_reward

In [2]:
import random
import numpy as np
import torch

TEST_NUM = 4
SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ENV_NAME = "grid2x2"

In [3]:
def train_env_creator(args):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(**env_params)
    return env

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

train_env = MultiAgentEnvCompatibility(train_env_creator({}))

config: PPOConfig
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_num_workers=1,
        evaluation_sample_timeout_s=300,
    )
    .debugging(seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "1")))
    .multi_agent(
        policies=set(train_env.env.ts_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .fault_tolerance(recreate_failed_workers=True)
)

delta_wait_time_reward 0.875


In [5]:
config.recreate_failed_workers

True

In [6]:
csv_dir = os.path.join("outputs",ENV_NAME,f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

## Play Untrained Agent

In [7]:
def eval_env_creator(csv_path, tb_log_dir):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": Grid2x2ObservationFunction,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(eval=True, csv_path=csv_path, tb_log_dir=tb_log_dir, **env_params)
    return env

In [8]:
ray.init()

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_untrained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

2023-06-03 18:30:47,651	INFO worker.py:1625 -- Started a local Ray instance.


In [9]:
algo = config.build()

2023-06-03 18:30:51,012	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-03 18:30:51,041	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=31392) delta_wait_time_reward 0.875
(RolloutWorker pid=31392) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=25080) 2023-06-03 18:30:57,540	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-03 18:31:01,228	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=24840) delta_wait_time_reward 0.875 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=19864) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 9x across cluster]


(RolloutWorker pid=24840) 2023-06-03 18:31:05,150	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-03 18:31:05,238	INFO trainable.py:172 -- Trainable.setup took 14.202 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [10]:
algo.evaluate()

Step #3600.00 (1ms ~= 1000.00*RT, ~223000.00UPS, TraCI: 241ms, vehicles TOT 1282 ACT 223 BOT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~223000.00UPS, TraCI: 241ms, vehicles TOT 1282 ACT 223 BOT 3 ACT 3 BUF 0)                      


{'evaluation': {'episode_reward_max': -491720.83761799976,
  'episode_reward_min': -491720.83761799976,
  'episode_reward_mean': -491720.83761799976,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -122972.34477694448,
   '2': -133132.50571310183,
   '5': -115436.13247978245,
   '6': -120179.85464817028},
  'policy_reward_max': {'1': -122972.34477694448,
   '2': -133132.50571310183,
   '5': -115436.13247978245,
   '6': -120179.85464817028},
  'policy_reward_mean': {'1': -122972.34477694448,
   '2': -133132.50571310183,
   '5': -115436.13247978245,
   '6': -120179.85464817028},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-491720.83761799976],
   'episode_lengths': [720],
   'policy_1_reward': [-122972.34477694448],
   'policy_2_reward': [-133132.50571310183],
   'policy_5_reward': [-115436.13247978245],
   'policy_6_reward': [-120179.85464817028]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 5.6353521413181

In [11]:
ray.shutdown()

## Train RL Agent

In [12]:
ray.init()

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(train_env_creator(config)))

2023-06-03 18:35:33,705	INFO worker.py:1625 -- Started a local Ray instance.


In [13]:
algo = config.build()

2023-06-03 18:35:37,131	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=5988) delta_wait_time_reward 0.875
(RolloutWorker pid=5988) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=8700) 2023-06-03 18:35:43,582	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-03 18:35:43,946	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-03 18:35:47,874	INFO trainable.py:172 -- Trainable.setup took 10.739 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [14]:
from datetime import datetime

TRAIN_EPS = 1400  # 720 * 1400 == 1_008_000 total timesteps
CHECKPOINT_FREQ = 100
assert TRAIN_EPS % CHECKPOINT_FREQ == 0

tic = datetime.now()

for i in range(TRAIN_EPS):
    results = algo.train()

    if (i+1) % CHECKPOINT_FREQ == 0:
        algo.save(os.path.join("ray_checkpoints","grid2x2",f"test_{TEST_NUM}"))

toc = datetime.now()

2023-06-03 18:36:01,343	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 109ms, vehicles TOT 2106 ACT 129 BUF 0)             OT 3 ACT 3 BUF 0)                      
(RolloutWorker pid=10040) delta_wait_time_reward 0.875 [repeated 10x across cluster]
(RolloutWorker pid=10040) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 105ms, vehicles TOT 2100 ACT 126 BUF 6)             OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~135000.00UPS, TraCI: 96ms, vehicles TOT 2094 ACT 135 BU27ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 92ms, vehicles TOT 2102 ACT 124 BUF 4)              T 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~182000.00UPS, TraCI: 104ms, vehicles TOT 2078 ACT 182 BT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 99ms, vehicles TOT 2101 ACT 181 BUF 5)              27ms, vehicles TOT 3 ACT 3 BUF 

(RolloutWorker pid=23532) Error: tcpip::Socket::accept() Unable to create listening socket: Address already in use
(RolloutWorker pid=23532) Quitting (on error).
(RolloutWorker pid=10040) Quitting (on error).
(RolloutWorker pid=23532) C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py:159: UserWarning: Could not connect to TraCI server using port 53298 (TraCI server already finished). Retrying with different port.
(RolloutWorker pid=23532)   warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


Step #3600.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 73ms, vehicles TOT 2106 ACT 44 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 73ms, vehicles TOT 2106 ACT 48 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 73ms, vehicles TOT 2106 ACT 44 BUF 0)               25ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 42 BUF 0)               0ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~49000.00UPS, TraCI: 73ms, vehicles TOT 2106 ACT 49 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 71ms, vehicles TOT 2106 ACT 42 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 58ms, vehicles TOT 2106 ACT 44 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 54ms, vehicles TOT 2106 ACT 50 BUF 0)               OT 3 

(RolloutWorker pid=29956) 2023-06-03 20:29:02,096	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=29956) Traceback (most recent call last):
(RolloutWorker pid=29956)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=29956)     return func(self, *args, **kwargs)
(RolloutWorker pid=29956)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=29956)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=29956)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=29956)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=29956)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

...

Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 68ms, vehicles TOT 2106 ACT 45 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 82ms, vehicles TOT 2106 ACT 49 BUF 0)               T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 87ms, vehicles TOT 2106 ACT 49 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 88ms, vehicles TOT 2106 ACT 44 BUF 16ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 89ms, vehicles TOT 2106 ACT 43 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 83ms, vehicles TOT 2106 ACT 42 BUF 0)               1ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~48000.00UPS, TraCI: 78ms, vehicles TOT 2106 ACT 48 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 72ms, vehicles TOT 2106 ACT 46 BUF 0)               T 3 

(RolloutWorker pid=23532) 2023-06-03 23:41:03,478	ERROR actor_manager.py:179 -- Worker exception, recreating: unpack requires a buffer of 3 bytes
(RolloutWorker pid=23532) Traceback (most recent call last):
(RolloutWorker pid=23532)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\utils\actor_manager.py", line 176, in apply
(RolloutWorker pid=23532)     return func(self, *args, **kwargs)
(RolloutWorker pid=23532)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 86, in <lambda>
(RolloutWorker pid=23532)     lambda w: w.sample(), local_worker=False, healthy_only=True
(RolloutWorker pid=23532)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
(RolloutWorker pid=23532)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=23532)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\rllib\evaluation\rollout_worker.py", line 

(RolloutWorker pid=17564) delta_wait_time_reward 0.875
(RolloutWorker pid=17564) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 63ms, vehicles TOT 2106 ACT 45 BUF 0)               T 3 ACT 3 BUF 0)                      
(RolloutWorker pid=21540) delta_wait_time_reward 0.875 [repeated 2x across cluster]
(RolloutWorker pid=21540) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 2x across cluster]
Step #3600.00 (1ms ~= 1000.00*RT, ~45000.00UPS, TraCI: 77ms, vehicles TOT 2106 ACT 45 BUF 5ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (1ms ~= 1000.00*RT, ~44000.00UPS, TraCI: 79ms, vehicles TOT 2106 ACT 44 BUF T 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 73ms, vehicles TOT 2106 ACT 43 BUF 0)               0ms, vehicles TOT 3 ACT 3 BUF 0)      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 78ms, vehicles TOT 2106 ACT 45 BUF 0)         

(RolloutWorker pid=17296) 2023-06-03 23:42:03,487	ERROR worker.py:844 -- Worker exits with an exit code 1. [repeated 2x across cluster]
(RolloutWorker pid=17296)     sys.exit(1) [repeated 6x across cluster]
(RolloutWorker pid=17296) During handling of the above exception, another exception occurred: [repeated 2x across cluster]
(RolloutWorker pid=17296)   File "python\ray\_raylet.pyx", line 1197, in ray._raylet.task_execution_handler [repeated 2x across cluster]
(RolloutWorker pid=17296)   File "python\ray\_raylet.pyx", line 1100, in ray._raylet.execute_task_with_cancellation_handler [repeated 2x across cluster]
(RolloutWorker pid=17296)   File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task [repeated 8x across cluster]
(RolloutWorker pid=17296)   File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor [repeated 2x across cluster]
(RolloutWorker pid=17296)   File "c:\Users\admin\anaconda3\envs\marl\lib\site-packages\ray\_private\function_m

(RolloutWorker pid=28768) delta_wait_time_reward 0.875
(RolloutWorker pid=28768) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 0 ACT 0 BUF 0)                      
Step #3600.00 (1ms ~= 1000.00*RT, ~49000.00UPS, TraCI: 79ms, vehicles TOT 2106 ACT 49 BUF OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 67ms, vehicles TOT 2106 ACT 41 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 76ms, vehicles TOT 2106 ACT 43 BUF 0)               OT 3 ACT 3 BUF 0)                      
Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 63ms, vehicles TOT 2106 ACT 43 BUF 0)               T 3 ACT 3 BUF 0)                      


In [16]:
toc - tic

datetime.timedelta(seconds=19685, microseconds=158906)

In [17]:
ray.shutdown()

## Play Trained Agent

In [18]:
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_trained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure",])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

In [19]:
ray.init()

2023-06-04 00:03:57,150	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.4.0


In [20]:
from ray.rllib.algorithms.ppo import PPO

checkpoint_path = os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}",f"checkpoint_{TRAIN_EPS:06}")
checkpoint_path = os.path.abspath(checkpoint_path)
print(checkpoint_path)

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-06-04 00:04:00,642	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_4\checkpoint_001400! Trying to extract checkpoint info from other files found in that dir.
2023-06-04 00:04:00,666	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


c:\Users\admin\masters-proj\ray_checkpoints\grid2x2\test_4\checkpoint_001400
(RolloutWorker pid=7336) delta_wait_time_reward 0.875
(RolloutWorker pid=7336) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 0 ACT 0 BUF 0)                      


(RolloutWorker pid=22868) 2023-06-04 00:04:07,423	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-04 00:04:07,700	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-04 00:04:11,654	INFO trainable.py:172 -- Trainable.setup took 10.984 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [21]:
ppo_agent.evaluate()

Step #3600.00 (0ms ?*RT. ?UPS, TraCI: 153ms, vehicles TOT 2106 ACT 46 BUF 0)              17ms, vehicles TOT 3 ACT 3 BUF 0)      
(RolloutWorker pid=29908) delta_wait_time_reward 0.875 [repeated 10x across cluster]
(RolloutWorker pid=29908) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                       [repeated 10x across cluster]


{'evaluation': {'episode_reward_max': -82058.44848137232,
  'episode_reward_min': -82058.44848137232,
  'episode_reward_mean': -82058.44848137232,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'1': -20795.213238814522,
   '2': -20457.03127213071,
   '5': -20061.660159948467,
   '6': -20744.543810478845},
  'policy_reward_max': {'1': -20795.213238814522,
   '2': -20457.03127213071,
   '5': -20061.660159948467,
   '6': -20744.543810478845},
  'policy_reward_mean': {'1': -20795.213238814522,
   '2': -20457.03127213071,
   '5': -20061.660159948467,
   '6': -20744.543810478845},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [-82058.44848137232],
   'episode_lengths': [720],
   'policy_1_reward': [-20795.213238814522],
   'policy_2_reward': [-20457.03127213071],
   'policy_5_reward': [-20061.660159948467],
   'policy_6_reward': [-20744.543810478845]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 5.619431996973808,
   '

In [22]:
ray.shutdown()

(RolloutWorker pid=29908) 2023-06-04 00:04:11,543	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
